In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from MTGpred.utils.database import get_all_decks,get_all_tournaments

db_decks = get_all_decks()

cards_distribution = {}

for deck in db_decks:
    for card in deck["main_deck"]:
        if card["name"] in cards_distribution:
            cards_distribution[card["name"]] += int(card["quantity"])
        else:
            cards_distribution[card["name"]] = int(card["quantity"])

In [10]:
sorted_cards_distribution = {k: v for k, v in sorted(cards_distribution.items(), key=lambda item: -int(item[1]))}

{'Island': 42532,
 'Mountain': 39417,
 'Forest': 39204,
 'Plains': 32037,
 'Swamp': 30581,
 'Force of Will': 22425,
 'Lightning Bolt': 17842,
 'Snow-Covered Island': 16456,
 'Scalding Tarn': 16202,
 'Brainstorm': 15991,
 'Preordain': 15918,
 'Polluted Delta': 14733,
 'Wasteland': 14435,
 'Ponder': 14257,
 'Flooded Strand': 14050,
 'Misty Rainforest': 13413,
 'Thoughtseize': 9249,
 'Volcanic Island': 9184,
 'Underground Sea': 7884,
 'Counterspell': 7562,
 'Fatal Push': 7060,
 'Snapcaster Mage': 7034,
 'Bloodstained Mire': 6942,
 'Daze': 6789,
 'Ancient Tomb': 6702,
 'Fabled Passage': 6664,
 'Steam Vents': 6490,
 'Mental Misstep': 6466,
 'Delver of Secrets': 6393,
 'Verdant Catacombs': 6366,
 'Wooded Foothills': 5953,
 'Gitaxian Probe': 5882,
 'Snow-Covered Forest': 5799,
 'Windswept Heath': 5448,
 'Hallowed Fountain': 5379,
 'Snow-Covered Plains': 5277,
 'Walking Ballista': 5237,
 'Swords to Plowshares': 5206,
 'Force of Negation': 5008,
 'Teferi, Time Raveler': 4965,
 'Expressive Itera

In [12]:
db_tournaments = get_all_tournaments()

formats_distribution = {}

for tournament in db_tournaments:
    if tournament["format"] in formats_distribution:
        formats_distribution[tournament["format"]] += 1
    else:
        formats_distribution[tournament["format"]] = 1

In [13]:
print(formats_distribution)

{'modern': 424, 'sealed': 285, 'standard': 315, 'pauper': 320, 'legacy': 267, 'pioneer': 221, 'other': 348, 'commander': 57, 'draft': 5}


In [9]:
from MTGpred.utils.database import get_matches_id_by_format,get_match
from MTGpred.model.dataset import MatchesDataset2
from MTGpred.utils.mtgjson import load_cards_df
from MTGpred.model.LSTM import WinnerPredictor
import torch

matches = get_matches_id_by_format("standard")
cards_df = load_cards_df("../data/AtomicCards.json")
dataset = MatchesDataset2(cards_df,matches,random_output=False)
model = WinnerPredictor()
model.load_state_dict(torch.load("../models/LSTM_best.pt"))
model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

WinnerPredictor(
  (encoder): DeckEncoder(
    (transformer): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0): TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm): La

In [10]:
def create_batch(batch_encoding):
    batch = {}
    for key in batch_encoding:
        batch[key] = batch_encoding[key].unsqueeze(0)
    return batch

for i,(deck1,deck2,mask1,mask2,target) in enumerate(dataset):
    deck1 = create_batch(deck1)
    deck2 = create_batch(deck2)

    match_id = dataset.get_match_id(i)
    match_data = get_match(match_id)
    print()
    print(match_data["p1_name"],match_data["p1_points"],"-",match_data["p2_name"],match_data["p2_points"])
    print(model(deck1,deck2,[mask1],[mask2]),target)
    print(model(deck2,deck1,[mask2],[mask1]))
    
    if i > 5:
        break